In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

Langchain_API_KEY=os.getenv("Langchain_API_KEY")

In [34]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
websites_strainers=[
    ["https://www.medkart.in/blog/5-ways-to-spot-harmful-ingredients-on-food-labels-or-read-food-labels-carefully",bs4.SoupStrainer(class_="ast-post-format- ast-no-thumb single-layout-1")],
   ["https://www.nhs.uk/live-well/eat-well/food-guidelines-and-food-labels/how-to-read-food-labels/",bs4.SoupStrainer(class_="nhsuk-grid-column-two-thirds")],
    ["https://www.acko.com/health-insurance/how-to-read-food-labels-for-nutritional-choices/",bs4.SoupStrainer(class_="sc-bdVaJa sc-bwzfXH sc-hEsumM hfMQBV")],
    # ["https://www.fda.gov/food/nutrition-facts-label/how-understand-and-use-nutrition-facts-label",bs4.SoupStrainer(class_="article main-content container-fluid")],
    ["https://timesofindia.indiatimes.com/life-style/health-fitness/diet/your-guide-to-reading-the-nutrition-label-right/articleshow/58622679.cms",bs4.SoupStrainer(class_="_s30J clearfix  ")]
]
all_articles=[]
for url,strainer in websites_strainers:
    loader = WebBaseLoader(
        web_paths=(url,),
        bs_kwargs={"parse_only": strainer},
    )
    article = loader.load()
    all_articles.append(article)

In [35]:
all_articles[3]


[Document(metadata={'source': 'https://timesofindia.indiatimes.com/life-style/health-fitness/diet/your-guide-to-reading-the-nutrition-label-right/articleshow/58622679.cms'}, page_content="A lot of us get a sense of confidence even by an attempt to have read the nutrition label that stays mostly misunderstood and half misinterpreted. If you want to count your calories, nutrition and sugars right, it is imperative that you know the clever marketing strategies used by food companies to make us believe we are eating healthy. The fancy terms like ‘brown sugar’ or ‘malt syrup’ are no less harmful; the terms like ‘natural’ and ‘energy-boosting’ are neither.If you want to be able to read a label right, you must know what's right and what's wrong. Lying somewhere in the middle of the two would be even worse. Here’s your guide to understanding a nutrition label the right way.Start with serving and caloriesThis will help you get your quantity right. The daily calorie intake for a healthy person s

In [58]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
)
article_splits = []
for article in all_articles:
    article_splits.append(text_splitter.split_documents(article))
print(type(article_splits[0][0]))


<class 'langchain_core.documents.base.Document'>


In [59]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
flattened_article_splits=[split for subsplit in article_splits for split in subsplit]
# print(type(flattened_article_splits[0]))
vector_store=FAISS.from_documents(flattened_article_splits,OllamaEmbeddings(model='mxbai-embed-large'))

In [60]:
len_vector_store = len(vector_store.index_to_docstore_id)
print(len_vector_store)

124


In [78]:
query = "Where are nutrition Labels?"
docs = vector_store.similarity_search_with_score(query,top_k=5)
print(type(docs[0][0]))
for doc in docs:
    print(f'''{doc[0].page_content} : Score={doc[1]} \n''')
vector_store.save_local("./data", "vector_store")

<class 'langchain_core.documents.base.Document'>
Nutrition information labels on the back or side of packagingMost pre-packed foods have a nutrition information label on the back or side of the packaging. These labels must include the amount of energy in kilojoules (kJ) and kilocalories (kcal), usually referred to as calories.They must also : Score=116.32060241699219 

sugar or salt.Some nutrition information labels on the back or side of packaging also provide information about reference intakes. : Score=117.79996490478516 

nutrition labels also provide information about reference intakes. : Score=118.61625671386719 

to read food labels to make informed nutritional choicesWhat is a nutrition facts label?A nutrition facts label is a set format that provides basic nutritional information about a food product. It typically includes information on calories, protein, total fat, saturated and trans fats, : Score=120.43757629394531 

